In [1]:
def orientation_evaluation(gt_pose, pred_rotmat, batch_size, curr_batch_size, step):

    import torch
    import numpy as np
    from scipy.spatial.transform import Rotation as R
    
    # Orientation evaluation
    # Taking as input gt_pose in axis-angle representation and pred_rotmat in rotation matrix representation

    gt_rotvec = torch.zeros((curr_batch_size,24,3), dtype=torch.double) # Reshaping the axis-angle (batch, 72) to (batch, 24, 3) for rotation vector compatibility

    for i, row in enumerate(gt_pose):
        gt_rotvec[i] = torch.reshape(row,(24, -1))

    #print("gt_rotvec", gt_rotvec.shape, gt_rotvec)

    # Get prediction as rotation vectors

    pred_rotvec_arr = np.zeros((curr_batch_size,24,3)) # Has to be a numpy array because it works with Rotation

    for i, row in enumerate(pred_rotmat):
        r = R.from_dcm(row.cpu()) # create the rotation object from the rotation matrix
        pred_rotvec_arr[i] = R.as_rotvec(r) # write it as rotation vectors in pred_rotvec_arr

    pred_rotvec = torch.from_numpy(pred_rotvec_arr) # transform it to a tensor

    #print("pred_rotvec", pred_rotvec.shape, pred_rotvec)

    orientation_error_per_part = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2))
    # This gives the error per part

    #print("error per part", orientation_error_non_reduced.shape, orientation_error_non_reduced)

    orientation_error = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2).sum(dim=-1).mean(dim=-1))
    # The reduction above is wrong. For a 90 degree error in one angle, it averages out 3.75 degrees, which
    # is 90/24. The correct reduction would be a mean of 1.25 (90/72), because there are 72 angles (3 for each part)
    # To remove the root, add [:,1:,:] to gt_euler and pred_euler above

    orientation_error_new = np.degrees(torch.sqrt((gt_rotvec - pred_rotvec)**2).mean(dim=[1,2]))
    # This reduction is more accurate because it averages the error per part and then the error across parts
    # It is equivalent to .mean(dim=-1).mean(dim=-1)

    #print(np.size(orientation_error_per_part), orientation_error_per_part)

    #print("orientation_error")
    #print(orientation_error)
    #print()
    #print("orientation_error_new")
    #print(orientation_error_new)
    #print()

    return orientation_error_per_part, orientation_error, orientation_error_new

In [2]:
"""
This script can be used to evaluate a trained model on 3D pose/shape and masks/part segmentation. You first need to download the datasets and preprocess them.
Example usage:
```
python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20
```
Running the above command will compute the MPJPE and Reconstruction Error on the Human3.6M dataset (Protocol I). The ```--dataset``` option can take different values based on the type of evaluation you want to perform:
1. Human3.6M Protocol 1 ```--dataset=h36m-p1```
2. Human3.6M Protocol 2 ```--dataset=h36m-p2```
3. 3DPW ```--dataset=3dpw```
4. LSP ```--dataset=lsp```
5. MPI-INF-3DHP ```--dataset=mpi-inf-3dhp```
"""

import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
import argparse
import json
from collections import namedtuple
from tqdm import tqdm
import torchgeometry as tgm

import config
import constants
from models import hmr, SMPL
from datasets import BaseDataset
from utils.imutils import uncrop
from utils.pose_utils import reconstruction_error
from utils.part_utils import PartRenderer

from scipy.spatial.transform import Rotation as R

# Define command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', default=None, help='Path to network checkpoint')
parser.add_argument('--dataset', default='h36m-p1', choices=['h36m-p1', 'h36m-p2', 'lsp', '3dpw', 'mpi-inf-3dhp'], help='Choose evaluation dataset')
parser.add_argument('--log_freq', default=50, type=int, help='Frequency of printing intermediate results')
parser.add_argument('--batch_size', default=32, help='Batch size for testing')
parser.add_argument('--shuffle', default=False, action='store_true', help='Shuffle data')
parser.add_argument('--num_workers', default=8, type=int, help='Number of processes for data loading')
parser.add_argument('--result_file', default=None, help='If set, save detections to a .npz file')

def run_evaluation(model, dataset_name, dataset, result_file,
                   batch_size=32, img_res=224, 
                   num_workers=32, shuffle=False, log_freq=50):
    """Run evaluation on the datasets and metrics we report in the paper. """

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Transfer model to the GPU
    model.to(device)

    # Load SMPL model
    smpl_neutral = SMPL(config.SMPL_MODEL_DIR,
                        create_transl=False).to(device)
    smpl_male = SMPL(config.SMPL_MODEL_DIR,
                     gender='male',
                     create_transl=False).to(device)
    smpl_female = SMPL(config.SMPL_MODEL_DIR,
                       gender='female',
                       create_transl=False).to(device)
    
    renderer = PartRenderer()
    
    # Regressor for H36m joints
    J_regressor = torch.from_numpy(np.load(config.JOINT_REGRESSOR_H36M)).float()
    
    save_results = result_file is not None
    # Disable shuffling if you want to save the results
    if save_results:
        shuffle=False
    # Create dataloader for the dataset
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    
    # Pose metrics
    # MPJPE and Reconstruction error for the non-parametric and parametric shapes
    mpjpe = np.zeros(len(dataset))
    recon_err = np.zeros(len(dataset))
    mpjpe_smpl = np.zeros(len(dataset))
    recon_err_smpl = np.zeros(len(dataset))
    
    #Including per joint position error:
    pjpe = torch.zeros(len(dataset), 14)
    
    # Including mean per joint angular error (reduced and per part)
    mpjae = np.zeros(len(dataset))
    mpjae_per_part = torch.zeros(len(dataset), 24, 3)

    # Shape metrics
    # Mean per-vertex error
    shape_err = np.zeros(len(dataset))
    shape_err_smpl = np.zeros(len(dataset))

    # Mask and part metrics
    # Accuracy
    accuracy = 0.
    parts_accuracy = 0.
    # True positive, false positive and false negative
    tp = np.zeros((2,1))
    fp = np.zeros((2,1))
    fn = np.zeros((2,1))
    parts_tp = np.zeros((7,1))
    parts_fp = np.zeros((7,1))
    parts_fn = np.zeros((7,1))
    # Pixel count accumulators
    pixel_count = 0
    parts_pixel_count = 0

    # Store SMPL parameters
    smpl_pose = np.zeros((len(dataset), 72))
    smpl_betas = np.zeros((len(dataset), 10))
    smpl_camera = np.zeros((len(dataset), 3))
    pred_joints = np.zeros((len(dataset), 17, 3))

    eval_pose = False
    eval_masks = False
    eval_parts = False
    eval_orientation = False # Adding the orientation parameter
    # Choose appropriate evaluation for each dataset
    if dataset_name == 'h36m-p1' or dataset_name == 'h36m-p2' or dataset_name == 'mpi-inf-3dhp':
        eval_pose = True
    elif dataset_name == 'lsp':
        eval_masks = True
        eval_parts = True
        annot_path = config.DATASET_FOLDERS['upi-s1h']
    elif dataset_name == '3dpw':
        eval_orientation = True
        eval_pose = True
        

    joint_mapper_h36m = constants.H36M_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.H36M_TO_J14
    joint_mapper_gt = constants.J24_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.J24_TO_J14
    # Iterate over the entire dataset
    for step, batch in enumerate(tqdm(data_loader, desc='Eval', total=len(data_loader))):
        # Get ground truth annotations from the batch
        gt_pose = batch['pose'].to(device)
        gt_betas = batch['betas'].to(device)
        gt_vertices = smpl_neutral(betas=gt_betas, body_pose=gt_pose[:, 3:], global_orient=gt_pose[:, :3]).vertices
        images = batch['img'].to(device)
        gender = batch['gender'].to(device)
        curr_batch_size = images.shape[0]
        
        with torch.no_grad():
            pred_rotmat, pred_betas, pred_camera = model(images)
            pred_output = smpl_neutral(betas=pred_betas, body_pose=pred_rotmat[:,1:], global_orient=pred_rotmat[:,0].unsqueeze(1), pose2rot=False)
            pred_vertices = pred_output.vertices

        if save_results:
            rot_pad = torch.tensor([0,0,1], dtype=torch.float32, device=device).view(1,3,1)
            rotmat = torch.cat((pred_rotmat.view(-1, 3, 3), rot_pad.expand(curr_batch_size * 24, -1, -1)), dim=-1)
            pred_pose = tgm.rotation_matrix_to_angle_axis(rotmat).contiguous().view(-1, 72)
            smpl_pose[step * batch_size:step * batch_size + curr_batch_size, :] = pred_pose.cpu().numpy()
            smpl_betas[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_betas.cpu().numpy()
            smpl_camera[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_camera.cpu().numpy()
        
        # Orientation evaluation
        orientation_error_per_part, orientation_error, orientation_error_new = \
        orientation_evaluation(gt_pose, pred_rotmat, batch_size, curr_batch_size, step)
        
        mpjae[step * batch_size:step * batch_size + curr_batch_size] = orientation_error_new
        mpjae_per_part[step*batch_size : step*batch_size + curr_batch_size] = orientation_error_per_part
            
        # 3D pose evaluation
        if eval_pose:
            # Regressor broadcasting
            J_regressor_batch = J_regressor[None, :].expand(pred_vertices.shape[0], -1, -1).to(device)
            # Get 14 ground truth joints
            if 'h36m' in dataset_name or 'mpi-inf' in dataset_name:
                gt_keypoints_3d = batch['pose_3d'].cuda()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_gt, :-1]
            # For 3DPW get the 14 common joints from the rendered shape
            else:
                gt_vertices = smpl_male(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices_female = smpl_female(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices[gender==1, :, :] = gt_vertices_female[gender==1, :, :]
                gt_keypoints_3d = torch.matmul(J_regressor_batch, gt_vertices) # torch.Size([32, 17, 3]) # This returns 17 joints
                gt_pelvis = gt_keypoints_3d[:, [0],:].clone()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_h36m, :] # torch.Size([32, 14, 3]) # But only 14 are used, the joint_mapper is [6, 5, 4, 1, 2, 3, 16, 15, 14, 11, 12, 13, 8, 10]
                
            # Get 14 predicted joints from the mesh
            pred_keypoints_3d = torch.matmul(J_regressor_batch, pred_vertices)
            if save_results:
                pred_joints[step * batch_size:step * batch_size + curr_batch_size, :, :]  = pred_keypoints_3d.cpu().numpy()
            pred_pelvis = pred_keypoints_3d[:, [0],:].clone()
            pred_keypoints_3d = pred_keypoints_3d[:, joint_mapper_h36m, :]
            pred_keypoints_3d = pred_keypoints_3d - pred_pelvis # [32, 14, 3]
            
            # Absolute error (MPJPE)
            error = torch.sqrt(((pred_keypoints_3d - gt_keypoints_3d) ** 2).sum(dim=-1)).mean(dim=-1).cpu().numpy()
            
            mpjpe[step * batch_size:step * batch_size + curr_batch_size] = error
            
            # Per part error
            per_part_error = torch.sqrt(((pred_keypoints_3d - gt_keypoints_3d) ** 2).sum(dim=-1)) # Not really necessary to send it to cpu as a np array for now
            
            pjpe[step * batch_size:step * batch_size + curr_batch_size] = per_part_error*1000 # Converting from meters to milimeters
            
            # Reconstuction_error
            r_error = reconstruction_error(pred_keypoints_3d.cpu().numpy(), gt_keypoints_3d.cpu().numpy(), reduction=None)
            recon_err[step * batch_size:step * batch_size + curr_batch_size] = r_error


        # If mask or part evaluation, render the mask and part images
        if eval_masks or eval_parts:
            mask, parts = renderer(pred_vertices, pred_camera)

        # Mask evaluation (for LSP)
        if eval_masks:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            # Dimensions of original image
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                # After rendering, convert imate back to original resolution
                pred_mask = uncrop(mask[i].cpu().numpy(), center[i], scale[i], orig_shape[i]) > 0
                # Load gt mask
                gt_mask = cv2.imread(os.path.join(annot_path, batch['maskname'][i]), 0) > 0
                # Evaluation consistent with the original UP-3D code
                accuracy += (gt_mask == pred_mask).sum()
                pixel_count += np.prod(np.array(gt_mask.shape))
                for c in range(2):
                    cgt = gt_mask == c
                    cpred = pred_mask == c
                    tp[c] += (cgt & cpred).sum()
                    fp[c] +=  (~cgt & cpred).sum()
                    fn[c] +=  (cgt & ~cpred).sum()
                f1 = 2 * tp / (2 * tp + fp + fn)

        # Part evaluation (for LSP)
        if eval_parts:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                pred_parts = uncrop(parts[i].cpu().numpy().astype(np.uint8), center[i], scale[i], orig_shape[i])
                # Load gt part segmentation
                gt_parts = cv2.imread(os.path.join(annot_path, batch['partname'][i]), 0)
                # Evaluation consistent with the original UP-3D code
                # 6 parts + background
                for c in range(7):
                   cgt = gt_parts == c
                   cpred = pred_parts == c
                   cpred[gt_parts == 255] = 0
                   parts_tp[c] += (cgt & cpred).sum()
                   parts_fp[c] +=  (~cgt & cpred).sum()
                   parts_fn[c] +=  (cgt & ~cpred).sum()
                gt_parts[gt_parts == 255] = 0
                pred_parts[pred_parts == 255] = 0
                parts_f1 = 2 * parts_tp / (2 * parts_tp + parts_fp + parts_fn)
                parts_accuracy += (gt_parts == pred_parts).sum()
                parts_pixel_count += np.prod(np.array(gt_parts.shape))

        # Print intermediate results during evaluation
        if step % log_freq == log_freq - 1:
            if eval_pose:
                print('MPJPE: ' + str(1000 * mpjpe[:step * batch_size].mean()))
                print('Reconstruction Error: ' + str(1000 * recon_err[:step * batch_size].mean()))
                print()
            if eval_masks:
                print('Accuracy: ', accuracy / pixel_count)
                print('F1: ', f1.mean())
                print()
            if eval_parts:
                print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
                print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
                print()
            if eval_orientation:
                print('Orientation error: ' + str(mpjae[:step * batch_size].mean()))

    # Save reconstructions to a file for further processing
    if save_results:
        np.savez(result_file, pred_joints=pred_joints, pose=smpl_pose, betas=smpl_betas, camera=smpl_camera)
    # Print final results during evaluation
    print('*** Final Results ***')
    print()
    if eval_pose:
        print('MPJPE: ' + str(1000 * mpjpe.mean()))
        print('Reconstruction Error: ' + str(1000 * recon_err.mean()))
        print()
        #torch.save(pjpe, 'pjpe.pt') # Uncomment to save the raw data to a file
    if eval_masks:
        print('Accuracy: ', accuracy / pixel_count)
        print('F1: ', f1.mean())
        print()
    if eval_parts:
        print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
        print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
        print()
    if eval_orientation:
        print('Orientation Error: ' + str(mpjae.mean()))
        #torch.save(mpjae_per_part, 'mpjae_per_part.pt') # Uncomment to save the raw data to a file

In [3]:
if __name__ == '__main__':
    
    # python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20 // example code
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--dataset=3dpw', '--log_freq=20'])
    # Here we inserted our own arguments list
    
    model = hmr(config.SMPL_MEAN_PARAMS)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)
    model.eval()

    # Setup evaluation dataset
    dataset = BaseDataset(None, args.dataset, is_train=False)
    # Run evaluation
    run_evaluation(model, args.dataset, dataset, args.result_file,
                   batch_size=args.batch_size,
                   shuffle=args.shuffle,
                   log_freq=args.log_freq)

Eval:   2%|▏         | 20/1110 [00:54<22:18,  1.23s/it]  

MPJPE: 258.302748472871
Reconstruction Error: 41.73969474351524

Orientation error: 8.301217391926139


Eval:   4%|▎         | 40/1110 [01:03<13:03,  1.37it/s]

MPJPE: 255.45146458376294
Reconstruction Error: 45.72069015538988

Orientation error: 8.917448099172002


Eval:   5%|▌         | 60/1110 [01:26<07:23,  2.37it/s]  

...

MPJPE: 263.85505667406863
Reconstruction Error: 59.155131115140506

Orientation error: 9.5267607877964


Eval:  99%|█████████▉| 1101/1110 [11:13<00:00,  9.25it/s]

MPJPE: 264.2032136099143
Reconstruction Error: 59.31143591334464

Orientation error: 9.518863738219308


Eval: 100%|██████████| 1110/1110 [11:14<00:00,  1.65it/s]

*** Final Results ***

MPJPE: 264.2195101033864
Reconstruction Error: 59.28574676292263

Orientation Error: 9.5149063225102


In [9]:
d = {
     'Left hip':{'Name': 'Left hip', 'x': 'Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right hip':{'Name': 'Right hip', 'x': 'Extension', 'y': 'Internal rotation', 'z': 'Adduction'},
     'Spine':{'Name': 'Spine', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Lateral flexion to the right'},
     'Left knee':{'Name': 'Left knee', 'x': 'Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right knee':{'Name': 'Right knee', 'x': 'Extension', 'y': 'External rotation', 'z': 'Adduction'},
     'Torso':{'Name': 'Torso', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Lateral flexion to the right'},
     'Left ankle':{'Name': 'Left ankle', 'x': 'Plantar Flexion', 'y': 'External rotation', 'z': 'Abduction'},
     'Right ankle':{'Name': 'Right ankle', 'x': 'Plantar Flexion', 'y': 'Internal rotation', 'z': 'Adduction'},
     'Chest':{'Name': 'Chest', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Lateral flexion to the right'},
     'Left toes':{'Name': 'Left toes', 'x': 'Flexion', 'y': 'Fibular deviation', 'z': 'Pronation'},
     'Right toes':{'Name': 'Right toes', 'x': 'Flexion', 'y': 'Tibial deviation', 'z': 'Supination'},
     'Neck':{'Name': 'Neck', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Lateral flexion to the right'},
     'Left scapula':{'Name': 'Left scapula', 'x': 'Elevation/protraction', 'y': 'Horizontal extension', 'z': 'Upwards rotation'},
     'Right scapula':{'Name': 'Right scapula', 'x': 'Elevation/protraction', 'y': 'Horizontal flexion', 'z': 'Downwards rotation'},
     'Skull':{'Name': 'Skull', 'x': 'Flexion', 'y': 'Rotate to the left', 'z': 'Lateral flexion to the right'},
     'Left shoulder':{'Name': 'Left shoulder', 'x': 'Internal rotation', 'y': 'Horizontal extension', 'z': 'Upwards rotation'},
     'Right shoulder':{'Name': 'Right shoulder', 'x': 'Internal rotation', 'y': 'Horizontal flexion', 'z': 'Downwards rotation'},
     'Left elbow':{'Name': 'Left elbow', 'x': 'Supination', 'y': 'Extension', 'z': 'Abduction'},
     'Right elbow':{'Name': 'Right elbow', 'x': 'Pronation', 'y': 'Flexion', 'z': 'Adduction'},
     'Left wrist':{'Name': 'Left wrist', 'x': 'Internal rotation', 'y': 'Ulnar deviation', 'z': 'Extension'},
     'Right wrist':{'Name': 'Right wrist', 'x': 'Internal rotation', 'y': 'Radial deviation', 'z': 'Flexion'},
     'Left knuckles':{'Name': 'Left knuckles', 'x': 'Internal rotation', 'y': 'Ulnar deviation', 'z': 'Extension'},
     'Right knuckles':{'Name': 'Right knuckles', 'x': 'Internal rotation', 'y': 'Radial deviation', 'z': 'Flexion'},
    }

# For the Spine, Torso, Chest, Neck and Skull, a rotational movement on the frontal plane is called a
# "Lateral flexion" to the right or left
# Rotation of the scapula happens about a sagittal axis, in the frontal plane, and can be defined as clockwise or
# counter-clockwise, or upwards/downwards
# Some of the joint movements are not anatomically described: raising the arm is mostly done by scapular
# upwards rotation, rather than upwards rotation on the shoulder joint.
# Plantar flexion happens when the feet ankles moves downwards
# Forearm external and internal rotation are, respectively, supination and pronation

In [12]:
# Angles defining the "Axial" orientation of the body parts:

axial_x = [
    'Left shoulder',
    'Right shoulder',
    'Left elbow',
    'Right elbow',
    'Left wrist',
    'Right wrist',
    'Left knuckles',
    'Right knuckles'
]

axial_y = [
    'Left hip',
    'Right hip',
    'Spine',
    'Left knee',
    'Right knee',
    'Torso',
    'Left ankle',
    'Right ankle',
    'Chest',
    'Neck',
    'Skull'
]

axial_z = [
    'Left toes',
    'Right toes',
    'Left scapula',
    'Right scapula',
]


# Printing the evaluated angles per part

e = mpjae_mean_no_root

i = 0
for key in d:
    if key in axial_x:
        print('{:16}'.format(d[key]['Name']),
              '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]))
    if key in axial_y:
        print('{:16}'.format(d[key]['Name']),
              '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]))
    if key in axial_z:
        print('{:16}'.format(d[key]['Name']),
              '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip            External rotation    5.62
Right hip           Internal rotation    4.61
Spine              Rotate to the left    2.08
Left knee           External rotation    5.13
Right knee          External rotation    4.13
Torso              Rotate to the left    2.16
Left ankle          External rotation    8.33
Right ankle         Internal rotation   11.17
Chest              Rotate to the left    1.69
Left toes                   Pronation    8.11
Right toes                 Supination   11.55
Neck               Rotate to the left    8.01
Left scapula         Upwards rotation   10.57
Right scapula      Downwards rotation   10.80
Skull              Rotate to the left    6.65
Left shoulder       Internal rotation    8.75
Right shoulder      Internal rotation   13.82
Left elbow                 Supination   16.63
Right elbow                 Pronation   15.89
Left wrist          Internal rotation    8.48
Right wrist         Internal rotation    5.16
Left knuckles       Internal rotat

In [3]:
import torch

mpjae = torch.load('mpjae_per_part.pt')
mpjae_mean = mpjae.mean(dim=0)
mpjae_mean_no_root = mpjae_mean[1:]
mpjae_mean_no_root.shape

torch.Size([23, 3])

In [4]:
# Printing the evaluated angles per part

e = mpjae_mean_no_root

i = 0
for key in d:    
    print('{:16}'.format(d[key]['Name']),
          '{:>20}'.format(d[key]['x']),'{:> 7.2f}'.format(e[i][0]),
          '{:>20}'.format(d[key]['y']),'{:> 7.2f}'.format(e[i][1]),
          '{:>20}'.format(d[key]['z']),'{:> 7.2f}'.format(e[i][2]))
    i+=1

Left hip                      Flexion   10.05    External rotation    5.62            Abduction    4.08
Right hip                   Extension    9.84    Internal rotation    4.61            Adduction    4.88
Spine                         Flexion    9.16   Rotate to the left    2.08    Touch right ankle    2.28
Left knee                     Flexion   16.95    External rotation    5.13            Abduction    6.10
Right knee                  Extension   16.49    External rotation    4.13            Adduction    4.92
Torso                         Flexion    4.42   Rotate to the left    2.16     Touch right shin    2.04
Left ankle                    Flexion    4.77    External rotation    8.33            Abduction    3.90
Right ankle                   Flexion    5.54    Internal rotation   11.17            Adduction    6.72
Chest                         Flexion    3.01   Rotate to the left    1.69     Touch right knee    1.01
Left toes                     Flexion   11.98    Fibular deviati